# Advent of Code - 2024 - Day 13 - Problem 2

https://adventofcode.com/2024/day/13

## Load Source Data

Load the map data into `DATA`.

In [1]:
f = open("data/day13.txt", "r")
DATA = list(map(str.strip, f.readlines()))
f.close()

## Create Game Class

Finds the number of presses by solving the following linear equations:

```
    A * ax + B * bx = px
    A * ay + B * by = py
```

This only works due to the nature of the input data (i.e., multiple solutions are not possible.)

See https://numpy.org/doc/2.1/reference/generated/numpy.linalg.solve.html

In [2]:
import numpy as np


class Game:

    def __init__(self, button_a_x, button_a_y, button_b_x, button_b_y, prize_x, prize_y):
        self._button_a_x = button_a_x
        self._button_a_y = button_a_y
        self._button_b_x = button_b_x
        self._button_b_y = button_b_y
        self._prize_x = prize_x
        self._prize_y = prize_y

    def __str__(self):
        return (
            f"_button_a_x = {self._button_a_x}, "
            + f"_button_a_y = {self._button_a_y}, "
            + f"_button_b_x = {self._button_b_x}, "
            + f"_button_b_y = {self._button_b_y}, "
            + f"_prize_x = {self._prize_x}, "
            + f"_prize_y = {self._prize_y}"
        )

    def play(self):

        coefficients = np.array([[self._button_a_x, self._button_b_x], [self._button_a_y, self._button_b_y]])
        results = np.array([self._prize_x, self._prize_y])
        solution = np.linalg.solve(coefficients, results)

        # Round to integers and ensure the results are valid (since buttons can't be pressed a "fractional" number of times.)
        #
        a_presses = round(solution[0])
        b_presses = round(solution[1])
        if (a_presses * self._button_a_x + b_presses * self._button_b_x == self._prize_x) and (
            a_presses * self._button_a_y + b_presses * self._button_b_y == self._prize_y
        ):
            return (a_presses, b_presses)

        return (-1, -1)

## Parse Data

Sample input:

```
Button A: X+97, Y+12
Button B: X+52, Y+63
Prize: X=10471, Y=4350
```

In [3]:
import re

GAMES = []

BUTTON_REGEX = re.compile("Button [AB]: X\+(\d+), Y\+(\d+)")
PRIZE_REGEX = re.compile("Prize: X=(\d+), Y=(\d+)")

idx = 0
while idx < len(DATA):
    button_a = BUTTON_REGEX.match(DATA[idx + 0])
    button_b = BUTTON_REGEX.match(DATA[idx + 1])
    prize = PRIZE_REGEX.match(DATA[idx + 2])

    button_a_x = int(button_a.group(1))
    button_a_y = int(button_a.group(2))
    button_b_x = int(button_b.group(1))
    button_b_y = int(button_b.group(2))
    prize_x = int(prize.group(1)) + 10000000000000
    prize_y = int(prize.group(2)) + 10000000000000

    g = Game(button_a_x, button_a_y, button_b_x, button_b_y, prize_x, prize_y)
    GAMES.append(g)
    idx += 4

## Play Games

In [4]:
token_count = 0

for game in GAMES:
    solution = game.play()

    a_presses, b_presses = solution
    if a_presses != -1 and b_presses != -1:
        token_count += (a_presses * 3) + b_presses

print(f"token_count = {token_count}")

token_count = 98080815200063
